In [ ]:
#Importing Libraries
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#Load the dataset
df=pd.read_csv("drive/My Drive/CSV_Files/DrugsDotCom_preprocessed.csv",encoding="ISO-8859-1")
df.head()

NameError: ignored

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [ ]:
#Droping duplicates
df=df.drop_duplicates(subset=["drugName","condition","review","rating"],keep='first')

In [ ]:
#resertting the index
df = df.copy()
df.reset_index(inplace = True)

In [ ]:
df1 = pd.DataFrame(df.groupby('drugName')['review'].apply(' '.join).reset_index())
df1.head()

,drugName,review
0,A + D Cracked Skin Relief,"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore."""
1,A / B Otic,"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief."""
2,Abacavir / lamivudine,"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ..."
3,Abacavir / lamivudine / zidovudine,"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since."""
4,Abatacept,"""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou..."


In [ ]:
df1['review_wordcount'] = df1['review'].str.split().str.len()

In [ ]:
sorted = df1.sort_values('review_wordcount', ascending=False)
sorted.head()

,drugName,review,review_wordcount
1514,Nexplanon,""" I have been on this for 2 1/2 years now. I am going to have it removed because I have gained weight using it, my periods are heavier and longer with clots, I have become very moody, anxious and angry at the very slightest thing which I wasnt before. I get very terrible migraines with aura. I spoke to my doctor and she said she does not think these side effects could be due to the nexplanon. ...",267069
854,Ethinyl estradiol / norethindrone,""" Im on my 2nd round of necon 1/35. I started taking it because the doctor thinks I have endometriosis, but no one is really sure. My periods are extremely painful and I lose my appetite completely. The side effects of necon are the same as symptoms of pregnancy which is kind of alarming. Im also even more depressed and anxious than I was before I started taking it. I take it without the place...",186537
1248,Levonorgestrel,""" I took levonorgestrel under the brand Postinor 2 after an incident with my boyfriend. My boyfriends semen got over my vaginal area and some of it did get inside me (shallow penetration) and this was during my ovulation period. Took the first pill in 43h, second pill 12h later (postinor requires you to eat 2 pills 12h apart). 1 week after taking the pills I had a lot of vaginal discharge and ...",158176
1940,Sertraline,"""(Sertraline) is the most worst depression treatment for my son. The first time when my son had started to take these pills he was feeling normal but one day he started to have unusual pains which he couldnt describe in the chest which was everyday. Because of the pain he was going to A&amp;E for a check up twice in a week and shockingly nothing had came up. But that pain took my precious sons...",149151
855,Ethinyl estradiol / norgestimate,""" Im 19 years old and started taking this birth control after my first one worked perfectly for a few months, until I began to get my period every 8 days. Ive been on this pill for about two weeks and I already hate it. Im constantly moody, depressed and over emotional. Its effecting my relationship between my family, friends and boyfriend. Doesnt help with acne, Ive never had acne till I bega...",147830


In [ ]:
df2 = df.groupby('drugName')['rating'].mean().reset_index()
df2.head()

,drugName,rating
0,A + D Cracked Skin Relief,10.00000
1,A / B Otic,10.00000
2,Abacavir / lamivudine,10.00000
3,Abacavir / lamivudine / zidovudine,9.00000
4,Abatacept,7.26087


In [ ]:
df3 = pd.DataFrame(df.groupby('drugName')['usefulCount'].mean().round().reset_index())
df3.head()

,drugName,usefulCount
0,A + D Cracked Skin Relief,6.0
1,A / B Otic,19.0
2,Abacavir / lamivudine,4.0
3,Abacavir / lamivudine / zidovudine,1.0
4,Abatacept,48.0


In [ ]:
data = pd.concat([df1.drugName,df1.review, df2.rating, df3.usefulCount, df1.review_wordcount], axis = 1)
data.head()

,drugName,review,rating,usefulCount,review_wordcount
0,A + D Cracked Skin Relief,"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore.""",10.00000,6.0,63
1,A / B Otic,"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief.""",10.00000,19.0,73
2,Abacavir / lamivudine,"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ...",10.00000,4.0,178
3,Abacavir / lamivudine / zidovudine,"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since.""",9.00000,1.0,41
4,Abatacept,"""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou...",7.26087,48.0,1603


In [ ]:
from textblob import TextBlob

In [ ]:
data['polarity'] = data['review'].apply(lambda x: TextBlob(x).sentiment.polarity)
data.head()

,drugName,review,rating,usefulCount,review_wordcount,polarity
0,A + D Cracked Skin Relief,"""I have severe cracked skin on my hands. Ive tried many different products - my skin is extremely sensitive. This product helps heal my skin, does not sting and is not greasy (important when using for hands). This is the best product Ive found for my condition, but it is hard to find - my drugstore cant even order it for me anymore.""",10.00000,6.0,63,0.284722
1,A / B Otic,"""It numbs the pain. It makes my ear feel heavier - but at least the throbbing subsides. I always feel drowsy after the dosage. With an ear infection - sleeping is good!"" ""These drops helped me so much. I was in severe pain for 8 days before I finally found a doctor who would prescribe them to me. I went to the ER 5 days in a row before I got any relief.""",10.00000,19.0,73,0.193750
2,Abacavir / lamivudine,"""Epzicom, Reyataz AND Norvir triple combination. It is expensive, but ALL hiv medications are expensive and there are programs to help pay for the medicine if you have a low wage since most of these HIV medicines cost between $3000 and $10000 or more. This one in combination with Reyataz and Norvir has brought my CD4 level up to 1100 and an undetectable viral load. I do have side effects from ...",10.00000,4.0,178,0.181250
3,Abacavir / lamivudine / zidovudine,"""I was diagnosed back in 2008 and have been on Trizivir since. My specialist mentioned it is not the newest medicine but it does the trick. One and a half years after starting it I was undetectable and have been since.""",9.00000,1.0,41,-0.055556
4,Abatacept,"""After rashes, pain, nausea, and elevated liver enzymes on Remicade, then Humira, the doc switched me to Orencia. Felt very tired the evening of the loading dose, but not ill. Now I feel better than I have felt in 4 years! LOVE this stuff!"" ""After the first week of using Orencia, I felt more energized. Before I would wake up feeling tired, and could barely move. I was stiff and could not go ou...",7.26087,48.0,1603,0.096676


In [ ]:
import string
def Effective_Review(Review):
    Pos_Word=[]
    Review=Review.split(".")
    for word in Review:
        tb = TextBlob(word)
        if tb.sentiment.polarity >= 0.7:
            Pos_Word.append(word)
    return Pos_Word

Effective = lambda x: Effective_Review(x)

In [ ]:
data['Pos_Words'] = data.review.apply(Effective)

In [ ]:
data.tail()

,drugName,review,rating,usefulCount,review_wordcount,polarity,Pos_Words
2330,Zyrtec-D,"""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,[]
2331,Zyvox,"""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"[ If this works then its an amazing, wonderful drug, It was however very effective]"
2332,ZzzQuil,"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,[]
2333,femhrt,"""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263,0.067333,"["" ""Femhrt worked wonderful, Works great]"
2334,pHisoHex,"""Before Phisohex could only be purchased by prescription, I used it regullarly for feminine hygenine. The product worked perfect, was long lasting, and I never had any side affects. I used Phisohex from 1966 until, much to my dismay, it was taken off my druggist over the counter shelves and could only be purchased with a prescription. Would still be using it if I could. Wonderful solution!""",10.000000,0.0,67,0.307143,"[ Wonderful solution!""]"


In [ ]:
def Not_Effective_Review(Review):
    Neg_Word=[]
    Review=Review.split(".")
    for word in Review:
        tb = TextBlob(word)
        if tb.sentiment.polarity <= -0.8:
            Neg_Word.append(word)
    return Neg_Word

Not_Effective = lambda x: Not_Effective_Review(x)

In [ ]:
data['Neg_Words'] = data.review.apply(Not_Effective)

In [ ]:
data.tail()

,drugName,review,rating,usefulCount,review_wordcount,polarity,Pos_Words,Neg_Words
2330,Zyrtec-D,"""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,[],[]
2331,Zyvox,"""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"[ If this works then its an amazing, wonderful drug, It was however very effective]","[ It is a very unpleasant feeling, It was the worst 2 weeks of my life, Weakness, I felt horrible, My doctor keeps on eye on me but the brown tongue and funky taste is disgusting]"
2332,ZzzQuil,"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,[],[]
2333,femhrt,"""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263,0.067333,"["" ""Femhrt worked wonderful, Works great]",[it worked well for symptom management but the side effect was horrible]
2334,pHisoHex,"""Before Phisohex could only be purchased by prescription, I used it regullarly for feminine hygenine. The product worked perfect, was long lasting, and I never had any side affects. I used Phisohex from 1966 until, much to my dismay, it was taken off my druggist over the counter shelves and could only be purchased with a prescription. Would still be using it if I could. Wonderful solution!""",10.000000,0.0,67,0.307143,"[ Wonderful solution!""]",[]


In [ ]:
#Trial

In [ ]:
side_effects = pd.read_csv('drive/My Drive/CSV_Files/sideeffects.csv', encoding = 'ISO-8859-1')
side_effects.head()

,# STITCH,Individual Side Effect,Side Effect Name
0,CID003062316,C1096328,central nervous system mass
1,CID003062316,C0162830,Photosensitivity reaction
2,CID003062316,C1611725,leukaemic infiltration brain
3,CID003062316,C0541767,platelet adhesiveness abnormal
4,CID003062316,C0242973,Ventricular dysfunction


In [ ]:
side_Effects=side_effects['Side Effect Name']

In [ ]:
side_Effects

0                      central nervous system mass
1                        Photosensitivity reaction
2                     leukaemic infiltration brain
3                   platelet adhesiveness abnormal
4                          Ventricular dysfunction
                            ...                   
174972               anterior chamber inflammation
174973                             splenic abscess
174974               hepatitis b antibody positive
174975    international normalised ratio increased
174976               Anterior chamber pigmentation
Name: Side Effect Name, Length: 174977, dtype: object

In [ ]:
def Side_Effect(Review):
    s_word=[]
    for w in side_Effects:
        if w in Review:
            s_word.append(w)
    
    s_word = list(set(s_word)) 
    return s_word
    

Sf = lambda x: Side_Effect(x)


In [ ]:
data['Side_Effect'] = data.review.apply(Sf)

In [ ]:
data.tail()

,drugName,review,rating,usefulCount,review_wordcount,polarity,Pos_Words,Neg_Words,Side_Effect
2330,Zyrtec-D,"""had tried claritin-d, and others prior to finding Zyrtec-D. I have now used it for a number of years, and find it difficult to breath without it. Works very well for me, while nothing else that I could find has."" ""This medicine works better than any other allergy medication with decongestion that I have ever tried.""",9.000000,13.0,56,0.015000,[],[],[]
2331,Zyvox,"""7 day treatment worked well on an infection that lasted over 5 months"" ""After having arthroscopic knee surgery, I developed a persistent skin infection (cellulitis) at the surgery site. I spent a month on different antibiotics that didnt clear the infection (cephalexin, clindamycin, and ciprofloxacin) and was finally admitted to the hospital for another surgery and treatment with IV antibioti...",8.714286,21.0,1494,-0.048765,"[ If this works then its an amazing, wonderful drug, It was however very effective]","[ It is a very unpleasant feeling, It was the worst 2 weeks of my life, Weakness, I felt horrible, My doctor keeps on eye on me but the brown tongue and funky taste is disgusting]","[open wound, Influenza, surgery, wound, Sepsis, bone infection]"
2332,ZzzQuil,"""Gave me rapid heart beats !!!!!!!!!"" ""I purchased the zzzquil and it did not work not 1 bit , I would not recommended medication to anyone..."" ""I usually work a 10 hr shift in the afternoon but my boss has changed my Saturday shift to 5am-1pm. We live on a farm and have 3 outside dogs that like to bark at night. The first morning shift I only got 2 hrs sleep because of our dogs. I purchased Z...",2.000000,7.0,139,0.078283,[],[],[]
2333,femhrt,"""femhrt took away my hot flashes within a week but the acne was unbearable. Stopped taking it."" ""Femhrt worked wonderful. Had to wait a couple of months for the full effect so give it time. The only downfall is that it is expensive and does not have a generic."" ""Hot flashes completely gone in less than 2 weeks. Works great."" ""I was on femHRT for four months and had BV (bacterial vaginitis) all...",6.500000,14.0,263,0.067333,"["" ""Femhrt worked wonderful, Works great]",[it worked well for symptom management but the side effect was horrible],"[bedridden, birth control]"
2334,pHisoHex,"""Before Phisohex could only be purchased by prescription, I used it regullarly for feminine hygenine. The product worked perfect, was long lasting, and I never had any side affects. I used Phisohex from 1966 until, much to my dismay, it was taken off my druggist over the counter shelves and could only be purchased with a prescription. Would still be using it if I could. Wonderful solution!""",10.000000,0.0,67,0.307143,"[ Wonderful solution!""]",[],[]


In [ ]:
data.to_csv("FinalModel.csv")